In [1]:
import numpy as np

In [2]:
class link:
    def __init__(self, name, a0, a1):
        self.name = name
        self.travel_time = 0 # the travel time of the link
        self._flow = 0 # the flow on the link
        self._paths = [] # the paths using this link
        self._a0 = a0
        self._a1 = a1
        
    def add_path(self, path):
        self._paths.append(path)
    
    def update_flow(self):
        self._flow = 0
        for path in self._paths:
            self._flow += path.flow
    
    def update_travel_time(self):
        self.update_flow()
        self.travel_time = self._a0 + self._a1 * self._flow
        
    def __str__(self):
        return self.name

In [3]:
class path:
    def __init__(self, name, links):
        self.name = name
        self._travel_time = 0 # the travel time of the paths
        self.flow = 0 # the flow on the paths
        self._links = links # the paths using this link
        for link in links:
            link.add_path(self)
    
    # add the marginal cost to see if you converge to the social optimum
    def get_travel_time(self):
        return self._travel_time
    
    def update_travel_time(self):
        self._travel_time = 0
        for link in self._links:
            self._travel_time += link.travel_time
            
    def __str__(self):
        return self.name

In [4]:
class network:
    def __init__(self, paths, links):
        self.paths = paths
        self.links = links
    
    def update(self):
        for link in self.links:
            link.update_travel_time()
        for path in self.paths:
            path.update_travel_time()
            
    def update_av(self, avs):
        for path in self.paths:
            path.flow = 0
        for av in avs:
            av.path.flow += av.flow
        self.update()
        for av in avs:
            av.give_reward()

In [5]:
ab = link('ab', 1, 1/100)
ac = link('ac', 2, 0)
bc = link('bc', 0.25, 0)
bd = link('bd', 2, 0)
cd = link('cd', 1, 1/100)

In [6]:
abcd = path('abcd', [ab, bc, cd])
abd = path('abd', [ab, bd])
acd = path('acd', [ac, cd])

In [7]:
braess = network([abd, acd, abcd], [ab, ac, bc, bd, cd])

In [8]:
braess.update()

In [9]:
#Nash equilibrium
abcd.flow = 50
abd.flow = 25
acd.flow = 25
braess.update()
print([path.get_travel_time() for path in braess.paths])

[3.75, 3.75, 3.75]


In [10]:
#social optimum
abcd.flow = 0
abd.flow = 50
acd.flow = 50
braess.update()
print([path.get_travel_time() for path in braess.paths])

[3.5, 3.5, 3.25]


In [11]:
class autonomous_vehicle:
    # p = 0.5
    def __init__(self, name, flow):
        self.name = name
        self.path = None
        self.reward = 0
        self.flow = flow
    
    def path_choice(self, path):
        if self.path == None:
            self.path = path
        # the following condition makes the system converges toward Nash
        # change this condition to make an faster convergence toward Nash
        p = abs(self.path.get_travel_time() - path.get_travel_time()) / self.path.get_travel_time()
        self.path = (path if np.random.rand() < p else self.path)
    
    def give_reward(self):
        # add here a NN which choose the path given the reward
        self.reward = - self.path.get_travel_time()

In [12]:
av1 = autonomous_vehicle('1', 25)
av2 = autonomous_vehicle('2', 25)
av3 = autonomous_vehicle('3', 25)
av4 = autonomous_vehicle('4', 25)

avs = [av1, av2, av3, av4]

In [13]:
for i in range(1000):
    best_path = [x for _,x in sorted([(path.get_travel_time(),path) for path in braess.paths],  key=lambda tup: tup[0])][0]
    #print([(str(path), path.flow, path.get_travel_time()) for path in braess.paths])
    #print(av2.p)
    for av in avs:
        av.path_choice(best_path)

    braess.update_av(avs)
    

In [14]:
print([str(av.path) for av in avs])
print([(str(path), path.flow, path.get_travel_time()) for path in braess.paths])

['acd', 'abcd', 'abd', 'abcd']
[('abd', 25, 3.75), ('acd', 25, 3.75), ('abcd', 50, 3.75)]


In [15]:
for av in avs:
    av.path = acd
braess.update_av(avs)